Definition boite autour de toulouse par exemple

In [1]:
from shapely.geometry import Polygon

xmin, ymin, xmax, ymax = 559000, 6253000, 589000, 6283000
bbox_toulouse = Polygon([
    (xmin, ymin),
    (xmax, ymin),
    (xmax, ymax),
    (xmin, ymax)
])



Chargement cadastre + reprojection

In [9]:
import geopandas as gpd

cadastre_path = "C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/cadastre/batiments.shp"
cadastre_gdf = gpd.read_file(cadastre_path)

# Vérification / reprojection en EPSG:2154
if cadastre_gdf.crs is None:
    pass

if cadastre_gdf.crs.to_string() != "EPSG:2154":
    cadastre_gdf = cadastre_gdf.to_crs("EPSG:2154")

c:\Users\manen\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/cadastre/batiments.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


liste des fichier .jp2 des bandes dans le dossier .safe de sentinelle

In [4]:
import os

# Par exemple :
safe_folder = r"C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/S2A_MSIL2A_20240827T105021_N0511_R051_T31TCJ_20240827T152946.SAFE"

r20m_folder = os.path.join(
    safe_folder,
    "GRANULE",
    "L2A_T31TCJ_A047953_20240827T105645",  # <-- à adapter selon votre granule
    "IMG_DATA",
    "R20m"
)

# Liste des bandes qu'on souhaite charger en 20m
bands_wanted = [
    "B01", "B02", "B03", "B04", 
    "B05", "B06", "B07", 
    "B8A", "B11", "B12"
]
# On exclut B09 et B10

# Construire la liste des fichiers .jp2 attendus
jp2_paths = {}
for band in bands_wanted:
    filename = f"T31TCJ_20240827T105021_{band}_20m.jp2"  # pattern
    for f in os.listdir(r20m_folder):
        if f.endswith(".jp2") and f"_{band}_" in f:
            fullpath = os.path.join(r20m_folder, f)
            jp2_paths[band] = fullpath
            break

print("Fichiers trouvés :")
for b, p in jp2_paths.items():
    print(b, "->", p)


Fichiers trouvés :
B01 -> C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/S2A_MSIL2A_20240827T105021_N0511_R051_T31TCJ_20240827T152946.SAFE\GRANULE\L2A_T31TCJ_A047953_20240827T105645\IMG_DATA\R20m\T31TCJ_20240827T105021_B01_20m.jp2
B02 -> C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/S2A_MSIL2A_20240827T105021_N0511_R051_T31TCJ_20240827T152946.SAFE\GRANULE\L2A_T31TCJ_A047953_20240827T105645\IMG_DATA\R20m\T31TCJ_20240827T105021_B02_20m.jp2
B03 -> C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/S2A_MSIL2A_20240827T105021_N0511_R051_T31TCJ_20240827T152946.SAFE\GRANULE\L2A_T31TCJ_A047953_20240827T105645\IMG_DATA\R20m\T31TCJ_20240827T105021_B03_20m.jp2
B04 -> C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/S2A_MSIL2A_20240827T105021_N0511_R051_T31TCJ_20240827T152946.SAFE\GRANULE\L2A_T31TCJ_A047953_20240827T105645\IMG_DATA\R20m\T31TCJ_20240827T105021_B04_20m.jp2
B05 -> C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/S2A_MSIL2A_20240827T105021_N0511_R051_T31TCJ_20240827T152946.SAFE\GRA

Dimmension d'une bande ici la bande 2

In [5]:
import rasterio
from rasterio import Affine
import numpy as np

# Choisissons B02 comme "bande de référence" (vous pouvez prendre une autre)
reference_band = "B02" 
ref_path = jp2_paths[reference_band]

with rasterio.open(ref_path) as ref_src:
    ref_profile = ref_src.profile.copy()
    ref_width = ref_profile["width"]
    ref_height = ref_profile["height"]
    ref_transform = ref_profile["transform"]
    ref_crs = ref_profile["crs"]

print("Référence : ", reference_band)
print("Dimension : ", ref_width, "x", ref_height)
print("CRS : ", ref_crs)


Référence :  B02
Dimension :  5490 x 5490
CRS :  EPSG:32631


Stack de nos 10 bandes en un tableau

In [6]:
wanted_bands = list(jp2_paths.keys())  # Par ex. ["B01","B02","B03",...,"B12"] (sans B9, B10)
wanted_bands.sort()  # Juste pour avoir un ordre cohérent (B01, B02,...)

nb_bands = len(wanted_bands)
stack_array = np.zeros((nb_bands, ref_height, ref_width), dtype=np.float32)

for i, band in enumerate(wanted_bands):
    band_file = jp2_paths[band]
    with rasterio.open(band_file) as band_src:
        # Vérifier si la taille est identique à la référence
        if (band_src.width != ref_width) or (band_src.height != ref_height):
            # Dans la plupart des cas, en R20m, tout sera identique.
            # Sinon, il faudrait re-projeter / re-échantillonner.
            raise ValueError(f"Dimension mismatch pour la bande {band}")
        
        band_data = band_src.read(1)  # lire la bande unique
        stack_array[i, :, :] = band_data

print("Taille du stack :", stack_array.shape)  
# => (Nombre_bandes, Height, Width)


Taille du stack : (10, 5490, 5490)


Exportation en .tif

In [9]:
out_profile = ref_profile.copy()
out_profile.update({
    "driver": "GTiff",
    "count": nb_bands,
    "dtype": "float32"
})

out_tiff_path = "sentinel_R20m_multi_bandes.tif"
with rasterio.open(out_tiff_path, "w", **out_profile) as dst:
    for i in range(nb_bands):
        dst.write(stack_array[i, :, :], i+1)  # i+1 = index de bande rasterio

print("Multi-bandes écrit :", out_tiff_path)


Multi-bandes écrit : sentinel_R20m_multi_bandes.tif


Reprojection de sentinelle en lambert 93

In [10]:
import rasterio
from rasterio.crs import CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

sentinel_path = "sentinel_R20m_multi_bandes.tif"  # À adapter

with rasterio.open(sentinel_path) as src:
    sentinel_crs = src.crs
    print("Projection Sentinel :", sentinel_crs)

    if sentinel_crs != CRS.from_epsg(2154):
        reprojected_path = "C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/SENTINEL2_L2A_EPSG2154.tif"
        
        transform, width, height = calculate_default_transform(
            src.crs, CRS.from_epsg(2154),
            src.width, src.height, *src.bounds
        )
        profile = src.profile.copy()
        profile.update({
            'crs': CRS.from_epsg(2154),
            'transform': transform,
            'width': width,
            'height': height
        })
        
        with rasterio.open(reprojected_path, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=CRS.from_epsg(2154),
                    resampling=Resampling.nearest
                )
        
        sentinel_path = reprojected_path  # on travaillera maintenant avec la version reprojetée

print("Fichier Sentinel final (EPSG:2154) :", sentinel_path)

Projection Sentinel : EPSG:32631
Fichier Sentinel final (EPSG:2154) : C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/SENTINEL2_L2A_EPSG2154.tif


On rogne le cadastre avec la bounding box

In [11]:
import geopandas as gpd
from shapely.geometry import Polygon

# Création d'un GeoDataFrame pour la bounding box
bbox_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:2154", geometry=[bbox_toulouse])

cadastre_toulouse_clipped = gpd.clip(cadastre_gdf, bbox_gdf)

# Ne garder que Polygone / MultiPolygone
cadastre_toulouse_poly = cadastre_toulouse_clipped[
    cadastre_toulouse_clipped.geometry.type.isin(["Polygon", "MultiPolygon"])
]

cadastre_toulouse_path = "C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/cadastre_toulouse.shp"
cadastre_toulouse_poly.to_file(cadastre_toulouse_path)


c:\Users\manen\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field created create as date field, though DateTime requested.
  ogr_write(
c:\Users\manen\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field updated create as date field, though DateTime requested.
  ogr_write(


In [3]:
sentinel_path="C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/SENTINEL2_L2A_EPSG2154.tif"

On rogne sentinelle avec la bounding box

In [4]:
import rasterio.mask

with rasterio.open(sentinel_path) as src:
    bbox_geom = [bbox_toulouse.__geo_interface__]
    
    try:
        out_image, out_transform = rasterio.mask.mask(
            src,
            bbox_geom,
            crop=True,
            all_touched=True,
            nodata=None
        )
    except ValueError:
        print("La bounding box ne recouvre pas le raster : aucune découpe possible.")
        # Vous pouvez soit faire un continue, un return, etc.
        # Dans ce cas, on sort de la fonction ou on passe à la tuile suivante.
        raise

    # Suite du code seulement si la découpe a fonctionné
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "count": out_image.shape[0],
    })
    
    sentinel_toulouse_path = "C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/SENTINEL2_Toulouse.tif"
    with rasterio.open(sentinel_toulouse_path, "w", **out_meta) as dst:
        dst.write(out_image)



Création de la grille de 500*500 sur la bounding box

In [12]:
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd

def generate_grid(bounds, cell_size=500, crs="EPSG:2154"):
    """
    Génère une grille de carrés de taille cell_size x cell_size
    couvrant la bounding box spécifiée.
    bounds : (xmin, ymin, xmax, ymax)
    cell_size : taille de la maille en mètres
    """
    xmin, ymin, xmax, ymax = bounds
    
    x_coords = np.arange(xmin, xmax, cell_size)
    y_coords = np.arange(ymin, ymax, cell_size)

    polygons = []
    for x in x_coords:
        for y in y_coords:
            x2 = x + cell_size
            y2 = y + cell_size
            poly = Polygon([(x, y), (x2, y), (x2, y2), (x, y2)])
            polygons.append(poly)
    
    grid_gdf = gpd.GeoDataFrame({"geometry": polygons}, crs=crs)
    return grid_gdf

# On récupère l'emprise (xmin, ymin, xmax, ymax) de la bounding box
(xmin, ymin, xmax, ymax) = bbox_toulouse.bounds
grid_500m = generate_grid((xmin, ymin, xmax, ymax), cell_size=500, crs="EPSG:2154")


Découpage en grille sentinel

In [13]:
import os
import rasterio.mask

sentinel_toulouse_path = "C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/SENTINEL2_Toulouse.tif"
output_dir = "C:/Users/manen/OneDrive/Bureau/S9 N7/BEI/donnees/tuiles_toulouse"
os.makedirs(output_dir, exist_ok=True)

with rasterio.open(sentinel_toulouse_path) as src:
    for idx, row in grid_500m.iterrows():
        tile_geom = [row.geometry.__geo_interface__]
        
        try:
            out_image, out_transform = rasterio.mask.mask(
                src,
                tile_geom,
                crop=True,
                all_touched=True,
                nodata=None
            )
        except ValueError:
            # Si la tuile n'intersecte pas du tout le raster, on passe
            continue
        
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        tile_id = f"tile_{idx:05d}"
        sentinel_tile_path = os.path.join(output_dir, f"{tile_id}_sentinel.tif")
        
        with rasterio.open(sentinel_tile_path, "w", **out_meta) as dst:
            dst.write(out_image)


rasterisation cadastre avec masque binaire

In [ ]:
from rasterio.features import rasterize
import numpy as np

def rasterize_cadastre(geom_list, out_shape, transform, fill=0, default_value=1):
    return rasterize(
        shapes=geom_list,
        out_shape=out_shape,
        transform=transform,
        fill=fill,
        all_touched=True,
        default_value=default_value,
        dtype=np.uint8
    )

cadastre_toulouse_gdf = gpd.read_file(cadastre_toulouse_path)

with rasterio.open(sentinel_toulouse_path) as src_global:
    pass  # on peut s'en servir pour la résolution, etc.

for idx, row in grid_500m.iterrows():
    tile_id = f"tile_{idx:05d}"
    
    sentinel_tile_path = os.path.join(output_dir, f"{tile_id}_sentinel.tif")
    if not os.path.exists(sentinel_tile_path):
        continue
    
    # Charger la tuile Sentinel pour récupérer meta, transform, etc.
    with rasterio.open(sentinel_tile_path) as src_tile:
        out_meta = src_tile.meta.copy()
        
        # Intersecter le cadastre avec la tuile
        tile_poly = gpd.GeoDataFrame(geometry=[row.geometry], crs="EPSG:2154")
        cadastre_clip = gpd.overlay(cadastre_toulouse_gdf, tile_poly, how='intersection')
        
        if cadastre_clip.empty:
            # Pas de parcelles dans cette tuile
            mask_arr = np.zeros((src_tile.height, src_tile.width), dtype=np.uint8)
        else:
            shapes = [(geom, 1) for geom in cadastre_clip.geometry]
            mask_arr = rasterize_cadastre(
                shapes,
                out_shape=(src_tile.height, src_tile.width),
                transform=src_tile.transform,
                fill=0,
                default_value=1
            )
        
        # Sauvegarder le masque
        out_meta.update({
            "count": 1,
            "dtype": "uint8"
        })
        cadastre_mask_path = os.path.join(output_dir, f"{tile_id}_cadastre_mask.tif")
        with rasterio.open(cadastre_mask_path, "w", **out_meta) as dst:
            dst.write(mask_arr, 1)
